# LangSmith Hands-on Tutorial

### Installing the Required Library

In [13]:
!pip install -U langsmith -q
!pip install langchain -q
!pip install openai -q
!pip install python-dotenv -q

### Load Environment Variables

In [9]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.environ.get('OPENAI_API_KEY')

print('OpenAI API Key Loaded: ', os.environ.get('OPENAI_API_KEY') is not None)

OpenAI API Key Loaded:  True


### Building Simple LLM Application

In [38]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
llm.predict("How is the weather in UAE?")

'The weather in the UAE can vary depending on the time of year. In general, the UAE has a hot desert climate with high temperatures year-round. The summers are extremely hot, with temperatures often exceeding 40 degrees Celsius (104 degrees Fahrenheit). The winters are milder, with temperatures ranging from 15-25 degrees Celsius (59-77 degrees Fahrenheit). Rainfall is minimal and mostly occurs during the winter months.\n\nOverall, the weather in the UAE is characterized by sunny skies, low humidity, and little precipitation, making it a popular destination for those seeking warm weather and sunshine.'

### Using LangSmith LangChain Tracer

In [20]:
from langchain.callbacks import LangChainTracer

tracer = LangChainTracer(project_name="LangChainTracerProject")
llm.predict("How many people live in UAE?",callbacks=[tracer])

'As of 2021, the estimated population of the United Arab Emirates (UAE) is around 9.9 million people.'

### Using Context Manager

In [23]:
from langchain.callbacks import tracing_v2_enabled

with tracing_v2_enabled(project_name="LangChainTracerProject"):
    llm.predict("How many people live in France?")

### Using Tags

In [40]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0, tags=["llm-tag"])
prompt = PromptTemplate.from_template("{input}")
chain = LLMChain(llm=llm, prompt=prompt, tags=["chain-tag","another-tag"])

chain("Hello, World!", tags=["shared-tags"])

{'input': 'Hello, World!', 'text': 'Hello! How can I assist you today?'}

### Using Groups 

In [43]:
from langchain.callbacks.manager import (trace_as_chain_group)

with trace_as_chain_group("ataya_group_name") as group_manager:
    pass

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm=ChatOpenAI(temperature=0.7)
prompt= PromptTemplate(
    input_variables=["question"],
    template="What is the answer to {question}"
)
chain = LLMChain(llm=llm, prompt=prompt)
with trace_as_chain_group("ataya_group_name") as group_manager:
    chain.run(question="What is the your name?", callbacks=group_manager)
    chain.run(question="What is your age?", callbacks=group_manager)
    chain.run(question="Where do you work?", callbacks=group_manager)

### Using Client class from LangSmith

In [58]:
from langsmith import Client

client = Client()
project_runs=client.list_runs(project_name="default")
project_runs

<generator object Client.list_runs at 0x12578d8a0>

In [60]:
from datetime import datetime, timedelta

today_runs = client.list_runs(
    project_name="default",
    start_time=datetime.now() - timedelta(days=1),
    run_type="llm"
)

today_runs

<generator object Client.list_runs at 0x12578d300>

In [62]:
for run in today_runs:
    print(run)

id=UUID('17a20712-1c15-4073-a386-22007cc5944b') name='ChatOpenAI' start_time=datetime.datetime(2024, 7, 2, 21, 42, 53, 724266) run_type='llm' end_time=datetime.datetime(2024, 7, 2, 21, 42, 54, 829655) extra={'invocation_params': {'model': 'gpt-3.5-turbo', 'model_name': 'gpt-3.5-turbo', 'stream': False, 'n': 1, 'temperature': 0.7, '_type': 'openai-chat', 'stop': None}, 'options': {'stop': None}, 'batch_size': 1, 'runtime': {'sdk': 'langsmith-py', 'sdk_version': '0.1.82', 'library': 'langsmith', 'platform': 'macOS-14.6-arm64-arm-64bit', 'runtime': 'python', 'py_implementation': 'CPython', 'runtime_version': '3.11.7', 'langchain_version': '0.1.20', 'langchain_core_version': '0.1.52', 'thread_count': 22.0, 'mem': {'rss': 280526848.0}, 'cpu': {'time': {'sys': 3.72275584, 'user': 8.425246208}, 'ctx_switches': {'voluntary': 127141.0, 'involuntary': 0.0}, 'percent': 0.0}}, 'metadata': {'revision_id': '9bb07c0-dirty', 'system_fingerprint': None}} error=None serialized=None events=[{'name': 'sta

### Using Metadata in LangSmith

In [71]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

chat_model=ChatOpenAI()
chain = LLMChain.from_string(llm= chat_model, template="What is the answer to {input}")

chain({"input":"What is the capital of France"}, metadata={"source":"interent"})

{'input': 'What is the capital of France',
 'text': 'The capital of France is Paris.'}

In [77]:
from langsmith import Client

client = Client()

runs = list(client.list_runs(
    project_name="default",
    filter='has(metadata, \'{"source":"interent"}\')',
))

print(list(runs))

[Run(id=UUID('e9723d54-9109-4344-840e-c584e396bbb2'), name='ChatOpenAI', start_time=datetime.datetime(2024, 7, 2, 22, 41, 46, 967724), run_type='llm', end_time=datetime.datetime(2024, 7, 2, 22, 41, 48, 117410), extra={'invocation_params': {'model': 'gpt-3.5-turbo', 'model_name': 'gpt-3.5-turbo', 'stream': False, 'n': 1, 'temperature': 0.7, '_type': 'openai-chat', 'stop': None}, 'options': {'stop': None}, 'batch_size': 1, 'metadata': {'source': 'interent', 'revision_id': '9bb07c0-dirty', 'system_fingerprint': None}, 'runtime': {'sdk': 'langsmith-py', 'sdk_version': '0.1.82', 'library': 'langsmith', 'platform': 'macOS-14.6-arm64-arm-64bit', 'runtime': 'python', 'py_implementation': 'CPython', 'runtime_version': '3.11.7', 'langchain_version': '0.1.20', 'langchain_core_version': '0.1.52', 'thread_count': 25.0, 'mem': {'rss': 304906240.0}, 'cpu': {'time': {'sys': 4.72084736, 'user': 12.724870144}, 'ctx_switches': {'voluntary': 206428.0, 'involuntary': 0.0}, 'percent': 0.0}}}, error=None, se

### Dataset and Evaluation in LangSmith

In [80]:
from langsmith import Client

client = Client()

dataset = client.upload_csv(
    csv_file="./Files/dataset.csv",
    input_keys=["Question"],
    output_keys=["Answer"],
    name="First CSV Dataset",
    description="Dataset with 5 questions and answers in CSV format",
    data_type="kv"
)

In [82]:
from langsmith import Client
from langchain.smith import RunEvalConfig, run_on_dataset

evaluation_config = RunEvalConfig(
    evaluators=[
        "qa",
        "context_qa",
        "cot_qa"
    ]
)

In [84]:
client = Client()
llm = ChatOpenAI()
run_on_dataset(
    dataset_name="First CSV Dataset",
    llm_or_chain_factory=llm,
    client=client,
    evaluation=evaluation_config,
    project_name="Evaluation Project"
)

View the evaluation results for project 'Evaluation Project' at:
https://smith.langchain.com/o/b1c74b64-69f9-5c8f-975c-ba13560c50a8/datasets/9c904d45-e5bd-43f2-bba1-ffa05329f517/compare?selectedSessions=f41c9566-5ab6-4ef1-9c7b-b072f690a337

View all tests for Dataset First CSV Dataset at:
https://smith.langchain.com/o/b1c74b64-69f9-5c8f-975c-ba13560c50a8/datasets/9c904d45-e5bd-43f2-bba1-ffa05329f517
[------------------------------------------------->] 5/5

{'project_name': 'Evaluation Project',
 'results': {'01ad7ef2-5050-48a4-9528-4a11724d3d29': {'input': {'Question': 'What is the key difference between North Indian and South Indian cuisine?'},
   'feedback': [EvaluationResult(key='correctness', score=1, value='CORRECT', comment='CORRECT', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('82d0d255-9a04-4cfc-ab9a-19a9439b9b03'))}, feedback_config=None, source_run_id=None, target_run_id=None),
    EvaluationResult(key='Contextual Accuracy', score=1, value='CORRECT', comment='CORRECT', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('fe973884-cb01-4526-8f17-5da93a94f562'))}, feedback_config=None, source_run_id=None, target_run_id=None),
    EvaluationResult(key='COT Contextual Accuracy', score=1, value='CORRECT', comment="The student's answer is in line with the context provided. The student correctly identifies that North Indian cuisine features more wheat-based dishes and richer gravies, and that South Indian c